In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

# Parámetros del sistema
m1, m2, m3, m4 = 6.0, 4.5, 4.5, 4.5  # masas (kg)
L1, L2, L3, L4 = 0.35, 0.30, 0.30, 0.15  # longitudes de los eslabones (m)
lc1, lc2, lc3, lc4 = L1 / 2, L2 / 2, L3 / 2, L4 / 2  # centro de masas
I1, I2, I3, I4 = 0.1, 0.1, 0.1, 0.1  # momentos de inercia
g = 9.81  # gravedad (m/s^2)

# Ganancias del controlador PD
Kp = np.diag([150, 150, 150, 150])  # ganancias proporcionales
Kd = np.diag([30, 30, 30, 30])  # ganancias derivativas

# Trayectoria deseada (convertir grados a radianes)
q_d = np.radians([0, -5.68, -102.33, 108.01])  # posiciones deseadas (rad)
qd_dot = np.zeros(4)  # velocidades deseadas (rad/s)
qd_ddot = np.zeros(4)  # aceleraciones deseadas (rad/s^2)

# Definir el modelo dinámico
def dinamica(t, y):
    q = y[:4]  # posiciones de las articulaciones
    q_dot = y[4:]  # velocidades de las articulaciones

    # Matriz de inercia
    H = np.diag([
        I1 + m1 * lc1**2,
        I2 + m2 * lc2**2,
        I3 + m3 * lc3**2,
        I4 + m4 * lc4**2
    ])

    # Fuerzas de Coriolis y centrífugas (simplificadas)
    C = np.zeros((4, 4))  # reemplazar con expresiones específicas si es necesario

    # Términos gravitacionales
    G = np.array([
        (m1 * lc1 + m2 * L1) * g * np.sin(q[0]),
        m2 * lc2 * g * np.sin(q[1]),
        m3 * lc3 * g * np.sin(q[2]),
        m4 * lc4 * g * np.sin(q[3]),
    ])

    # Control PD
    tau = Kp @ (q_d - q) + Kd @ (qd_dot - q_dot)

    # Calcular aceleraciones
    q_ddot = np.linalg.inv(H) @ (tau - C @ q_dot - G)

    return np.concatenate([q_dot, q_ddot])

# Condiciones iniciales
q0 = np.array([0, 0, 0, 0])  # posiciones iniciales de las articulaciones (rad)
q0_dot = np.zeros(4)  # velocidades iniciales de las articulaciones (rad/s)
y0 = np.concatenate([q0, q0_dot])

# Tiempo de simulación
t_span = (0, 10)  # simular durante 10 segundos
t_eval = np.linspace(t_span[0], t_span[1], 1000)

# Resolver el sistema
solucion = solve_ivp(dinamica, t_span, y0, t_eval=t_eval)

# Extraer resultados
t = solucion.t
q = solucion.y[:4, :]  # posiciones de las articulaciones
q_dot = solucion.y[4:, :]  # velocidades de las articulaciones

# Graficar resultados
plt.figure(figsize=(12, 6))
for i in range(4):
    plt.plot(t, q[i, :], label=f"q{i+1} (rad)")
    plt.axhline(y=q_d[i], color='k', linestyle='--', label=f"q{i+1} deseado" if i == 0 else "")
plt.xlabel("Tiempo (s)")
plt.ylabel("Posición (rad)")
plt.title("Posiciones de las articulaciones bajo control PD para la trayectoria dada")
plt.legend()
plt.grid()
plt.show()